## 导库

没有安装以下库的话，请先安装

In [1]:
# !pip install jieba
# !pip install plotly
# !pip install torch

In [2]:
import re
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import jieba
import plotly.express as px
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = 'TRUE'
jieba.load_userdict("./data/dict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xiaof\AppData\Local\Temp\jieba.cache
Loading model cost 0.377 seconds.
Prefix dict has been built successfully.


### 工具函数

In [3]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

## 数据预处理

为了不影响准确度，如果字符属于数字，就将其替换为数字特殊 token-
“[num]”，处理后句子变成“[num]年”；

如果字符属于其他语言，就将其替换为字母特殊 token-“[eng]”，处理后句子变成“[eng]”；


In [4]:
filtering_data_file = './data/MuCGEC_dev.txt'
filtered_data_file = './data/MuCGEC_dev_filtered.txt'

In [5]:
sentences = []
with open(filtering_data_file, 'r') as f:
    lines = f.readlines()
    # 去掉最前面的数字和制表符
    sentences = [line.split('\t', 1)[1] for line in lines]
    sentences = [sentence.split('\t') for sentence in sentences]
    eng_pattern = re.compile(r'[a-zA-Z]+')
    num_pattern = re.compile(r'\d+')
    # 去掉最后的换行符
    for i, sentence in enumerate(sentences):
        for j, s in enumerate(sentence):

            s = s.strip('\n').strip('。')
            # 将非中文部分的.替换为。
            sentence[j] = s.replace('.', '。').replace('?', '？').replace(
                '!', '！').replace(';', '；').replace(',', '，')

        sentences[i] = sentence
print(sentences[:10])

[['因为在冰箱里没什么东西也做很好吃的菜', '即使在冰箱里没什么东西也能做很好吃的菜', '即使在冰箱里没什么东西，也能做很好吃的菜'], ['再说，他们可能以为自己吸烟时对别人带来的不便和不舒服也不可能很大', '再说，他们可能以为自己吸烟时给别人带来的不便和不舒服也不会很大', '再说，他们可能以为自己吸烟时给别人带来的不便和不舒服也不是很大', '再说，他们可能以为自己吸烟时给别人带来的不便和不舒服也不可能很大'], ['这样我们在生活当中没有了跟男生接触的机会，总会有一天离开学校的，我们那时肯定会遇到到底怎样与男人交流、沟通的问题了', '这样我们在生活当中没有了跟男生接触的机会，但我们总会有一天要离开学校的，我们离开学校后肯定会遇到到底应该怎样与男人交流、沟通的问题', '这样我们在生活当中没有了跟男生接触的机会，等有一天离开学校的时候，我们肯定会遇到到底应该怎样与男人交流、沟通的问题', '这样我们在生活当中没有了跟男生接触的机会，我们总会有一天离开学校的，我们那时肯定会遇到到底怎样与男人交流、沟通的问题'], ['我不愿意这么过分的喜欢歌手', '我不愿意这么过分地喜欢歌手', '我不愿意这么过分痴迷地喜欢歌手'], ['学生大概做飞机去北京', '学生大概坐飞机去北京', '学生大概是坐飞机去北京'], ['今天我看见了我家周边的西药店，吊着牌子称；“口罩都卖光了”', '今天我看见我家周边的西药店，吊着牌子称：“口罩都卖光了。”', '今天我看见我家周边的西药店吊着牌子称：“口罩都卖光了。”'], ['我是中学生，不知因为我是喜爱流行、时尚的人，但我认为流行歌曲是一种自己享受快乐的一种方法', '我是中学生，不知是否是因为我是一个喜爱流行、时尚的人才会这么想，但我确实认为流行歌曲是一种自己享受快乐的方法', '我是中学生，不知是否是因为我是一个喜爱流行、时尚的人才会这么想，但我确实认为流行歌曲是自己享受快乐的一种方法'], ['改变自己的身体的冲动往往是一个深刻精神不稳定的症状。它应该被视为一个问题，而不是纵容还有鼓励美容手术治疗', '改变自己身体的冲动往往是一种深层次的精神不稳定的症状。它应该被视为是一个问题，而不是纵容、鼓励美容手术治疗', '改变自己的身体的冲动往往是一个精神极不稳定的症状。它应该被视为一个问题，而不是被纵容或者被鼓励采

每一个错误句子只能有一个正确句子  
如果一行只有一个句子，认为他是正确的，对应的正确句也就是他  
如果一行有多个句子，认为有多个正确句，遍历正确句，每一个正确句和错误句组成一个样本


In [6]:
filtered_sentences = []
for index, sentence in enumerate(sentences):
    if len(sentence) == 1:
        filtered_sentences.append([sentence[0], sentence[0]])
        continue
    elif len(sentence) == 2:
        if sentence[1] == '没有错误':
            sentence[1] = sentence[0]
        filtered_sentences.append(sentence)
        continue
    elif len(sentence) > 2:
        for i in range(1, len(sentence)):
            filtered_sentences.append([sentence[0], sentence[i]])
        filtered_sentences.pop(index)
    elif sentence[1].find('[缺失成分]') or sentence[1].find('[缺失成分]'):
        filtered_sentences.pop(index)
print(filtered_sentences[:10])

[['因为在冰箱里没什么东西也做很好吃的菜', '即使在冰箱里没什么东西，也能做很好吃的菜'], ['再说，他们可能以为自己吸烟时对别人带来的不便和不舒服也不可能很大', '再说，他们可能以为自己吸烟时给别人带来的不便和不舒服也不是很大'], ['这样我们在生活当中没有了跟男生接触的机会，总会有一天离开学校的，我们那时肯定会遇到到底怎样与男人交流、沟通的问题了', '这样我们在生活当中没有了跟男生接触的机会，但我们总会有一天要离开学校的，我们离开学校后肯定会遇到到底应该怎样与男人交流、沟通的问题'], ['这样我们在生活当中没有了跟男生接触的机会，总会有一天离开学校的，我们那时肯定会遇到到底怎样与男人交流、沟通的问题了', '这样我们在生活当中没有了跟男生接触的机会，我们总会有一天离开学校的，我们那时肯定会遇到到底怎样与男人交流、沟通的问题'], ['我不愿意这么过分的喜欢歌手', '我不愿意这么过分痴迷地喜欢歌手'], ['学生大概做飞机去北京', '学生大概是坐飞机去北京'], ['今天我看见了我家周边的西药店，吊着牌子称；“口罩都卖光了”', '今天我看见我家周边的西药店吊着牌子称：“口罩都卖光了。”'], ['我是中学生，不知因为我是喜爱流行、时尚的人，但我认为流行歌曲是一种自己享受快乐的一种方法', '我是中学生，不知是否是因为我是一个喜爱流行、时尚的人才会这么想，但我确实认为流行歌曲是自己享受快乐的一种方法'], ['改变自己的身体的冲动往往是一个深刻精神不稳定的症状。它应该被视为一个问题，而不是纵容还有鼓励美容手术治疗', '改变自己的身体的冲动往往是一个精神极不稳定的症状。它应该被视为一个问题，而不是被纵容或者被鼓励采用美容手术治疗'], ['这个巨大飞的物体停住了一会就又向天空快速地飞去。一闪而过，变成了一个形影一样的亮点，一点点地暗下去', '这个巨大的飞行物体停了一会儿就又向天空快速地飞去了。它一闪而过，变成了一个星星一样的亮点，然后一点点地暗下去']]


写入文件**filtered.txt中

In [7]:
with open(filtered_data_file, 'w') as f:
    for sentence in filtered_sentences:
        f.write(sentence[0] + '\t' + sentence[1] + '\n')

## 加载数据

In [8]:
train_data_file = './data/MuCGEC_dev_filtered.txt'  # 训练数据
test_data_file = './data/MuCGEC_test.txt'

### 辅助加载类

这个用来存储每一个词语的索引和单词哈希表

In [9]:
SOS_token = 0
EOS_token = 1
UNK = 2


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {
            "[SOS]": 0,
            "[EOS]": 1,
            "[UNK]": 2
        }
        self.word2count = {
            "[SOS]": 0,
            "[EOS]": 0,
            "[UNK]": 0
        }
        self.index2word = {0: "[SOS]", 1: "[EOS]", 2: "[UNK]"}
        # self.index2word = {}
        self.n_words = 3  # Count SOS and EOS and UNK

    def addSentence(self, sentence):
        for word in jieba.lcut(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

根据每段句子的长度，确定最大长度

In [10]:
with open(train_data_file, 'r') as f:
    lines = f.readlines()
    sentences = [[s for s in l.split(' ')] for l in lines]
# 计算每个句子的长度
sentence_lengths = [len(jieba.lcut(sentence_pair[0]))
                    for sentence_pair in sentences]

# 使用Plotly绘制直方图
fig = px.histogram(sentence_lengths, nbins=200, title='Sentence Length Distribution', labels={
                   'value': 'Sentence Length', 'count': 'Frequency'})
fig.show()

图中句子的最大长度在40-60之间更好

将数据集修剪为相对较短且简单的句子

In [11]:
MAX_LENGTH = 40


def filterPair(p):
    # 因为后面会加<EOS>，所以这里减去一些长度
    ret = len(jieba.lcut(p[0])) < MAX_LENGTH and len(
        jieba.lcut(p[1])) < MAX_LENGTH
    return ret

### 记录dev数据集词语

In [12]:
input_lang = Lang('input')
output_lang = Lang('output')
pairs = []
with open(train_data_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        pair = line.strip('\n').split(' ')
        if not filterPair(pair):
            # 如果长度不符合要求则跳过
            continue
        pairs.append(pair)
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

### 记录test数据集词语

In [13]:
test_sentences = []
with open(test_data_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        s = line.split('\t')[1]
        test_sentences.append(s.strip('\n'))
test_sentences[:5]

['冬阴功是泰国最著名的菜之一，它虽然不是很豪华，但它的味确实让人上瘾，做法也不难、不复杂。',
 '首先，我们得准备：大虾六到九只、盐一茶匙、已搾好的柠檬汁三汤匙、泰国柠檬叶三叶、柠檬香草一根、鱼酱两汤匙、辣椒6粒，纯净水4量杯、香菜半量杯和草菇10个。',
 '这样，你就会尝到泰国人死爱的味道。',
 '另外，冬阴功对外国人的喜爱不断地增加。',
 '这部电影不仅是国内，在国外也很有名。']

In [14]:
filtered_test_sentences = []
for sentence in test_sentences:
    cut_s = jieba.lcut(sentence, HMM=False)
    if len(cut_s) < MAX_LENGTH:
        filtered_test_sentences.append(sentence)
for sentence in filtered_test_sentences:
    words = jieba.lcut(sentence, HMM=False)
    for w in words:
        if w not in input_lang.word2index.keys():
            input_lang.addWord(w)
        if w not in output_lang.word2index.keys():
            output_lang.addWord(w)

In [15]:
print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)
print(random.choice(pairs))

Counted words:
input 12800
output 12778
['特别高中学生不能控制自己，常常犯了错误', '特别是高中学生，不能控制自己，常常犯错误']


## 构建模型

### 定义编码器

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

### 定义解码器

### 构建注意力机制

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)  # 获取第一个维度大小
        decoder_input = torch.empty(
            batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)
            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(
                    1)  # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                # detach from history as input
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

## 训练模型

In [18]:
def indexesFromSentence(lang, sentence):
    temp_list = []
    for word in jieba.lcut(sentence):
        temp_list.append(lang.word2index.get(word, lang.word2index['[UNK]']))
    return temp_list


def tensorFromSentence(lang, sentence):
    # 将句子转换为索引的Tensor
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


def get_dataloader(batch_size):
    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)

        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(
        train_data, sampler=train_sampler, batch_size=batch_size)
    return train_dataloader

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
                decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(
            encoder_outputs, encoder_hidden, target_tensor)
        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
          print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    encoder.train()
    decoder.train()

    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder,
                           encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                         epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

### 训练启动

In [26]:
hidden_size = 256
batch_size = 64
n_epochs = 100

train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder,
      n_epochs, print_every=1, plot_every=1)

torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([64, 40, 256])
torch.Size([

KeyboardInterrupt: 

### 保存模型

In [ ]:
model_save_path = './data/seq2seq.pth'
torch.save({
    'encoder': encoder.state_dict(),
    'decoder': decoder.state_dict(),
    'input_lang': input_lang,
    'output_lang': output_lang
}, model_save_path)

## 评估模型

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(
            encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(
            encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence.replace(' ', '').replace('<EOS>', ''))
        print('')

### 评估启动

In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> 我没有够朋友
= 我没有交够朋友
< 我没有足够的人没想过

> 我哥哥是大学生。他每天在大学学心理学。他很喜欢读心理学的书惟其他患精神分裂症
= 我哥哥是大学生。他每天在大学学心理学。他很喜欢读心理学的书导致他患精神分裂症
< 我哥哥是大学生，他每天在大学里学心理学，他很喜欢读心理学的书，我唯恐他患精神分裂症

> 我老公今年出差不多，所以可以参观了孩子的运动会什么等
= 我老公今年出差不多，所以可以参加孩子的运动会什么的
< 我老公今年出差不多，所以可以参加孩子的运动会等活动。因为孩子的运动会等活动

> 有一个小女孩，她非常喜欢香港的一位女歌手那个小女孩，她也想做歌手，可是她认为“我太胖了”过分的减肥，她终于死了
= 有一个小女孩，她非常喜欢香港的一位女歌手，那个小女孩也想做歌手，可是她认为自己太胖了而过分地减肥，最后去世了
< 有一个小女孩，她非常喜欢香港的一位女歌手，那个小女孩也想做歌手，可是她认为自己太胖了而过分地减肥，最后去世了

> 这次大地震的时候，岛里的灯塔和神社入口的牌坊都被冲到海里
= 这次大地震的时候，岛里的灯塔和神社入口的牌坊都被冲到了海里
< 这次大地震的时候，岛里的灯塔和神社入口的牌坊都被冲到了海里

> 父母平时在家的相处或对于长辈之间的态度，在孩子眼中都是能影响他们一生的关键
= 父母之间平时在家的相处情况和对长辈的态度，对孩子来说能影响他们的一生
< 父母之间平时在家的相处情况和对长辈的态度，对孩子来说能影响他们的一生

> 吸烟者觉得没有吸烟得话不能休息精神
= 吸烟者觉得没有吸烟的话不能放松精神
< 吸烟者觉得不吸烟的话不能放松精神不能放松精神

> 军队能让年轻人成熟起来吗？对于我观点有凭证：我觉得是不可能的
= 军队能让年轻人成熟起来吗？对于这点，我观点是：这是不可能的
< 军队能让年轻人成熟起来吗？对于我的观点我有凭证，我觉得是不可能的

> 所以我从小到现在在这些快餐吃饭的机会很少。对我来说每次饭都很重要
= 所以我从小到现在在这些快餐店吃饭的机会很少。对我来说每次吃饭都很重要
< 我从小到现在在这些快餐店吃饭的机会很少，所以对我来说每顿饭都很重要。我都很重要都很重要

> 我爱好很好。例如，我喜欢读书。终于，我爱打篮球。每星期一和星期二我下课的时候坐公共汽车去日内瓦大学的体育馆打篮球
= 我爱好很多。例如，我喜

## 预测测试集

### 清理测试集

In [ ]:
test_sentences = []
with open(test_data_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        s = line.split('\t')[1]
        test_sentences.append(s.strip('\n'))
test_sentences[:5]

['冬阴功是泰国最著名的菜之一，它虽然不是很豪华，但它的味确实让人上瘾，做法也不难、不复杂。',
 '首先，我们得准备：大虾六到九只、盐一茶匙、已搾好的柠檬汁三汤匙、泰国柠檬叶三叶、柠檬香草一根、鱼酱两汤匙、辣椒6粒，纯净水4量杯、香菜半量杯和草菇10个。',
 '这样，你就会尝到泰国人死爱的味道。',
 '另外，冬阴功对外国人的喜爱不断地增加。',
 '这部电影不仅是国内，在国外也很有名。']

### 预测

In [ ]:
encoder.eval()
for sentence in filtered_test_sentences:
    output_words, _ = evaluate(
        encoder, decoder, sentence, input_lang, output_lang)
    output_sentence = ' '.join(output_words)
    print(sentence)
    print(output_sentence.replace(' ', '').replace('<EOS>', ''))
    print('')

冬阴功是泰国最著名的菜之一，它虽然不是很豪华，但它的味确实让人上瘾，做法也不难、不复杂。
欧洲是最重要的，而且歌曲觉得不的时候，并且也不能一辈子好的背影，就会再次认真。即使心情不需要

这样，你就会尝到泰国人死爱的味道。
这样，你会帮助就会受“怎样做人的结局

另外，冬阴功对外国人的喜爱不断地增加。
第二，对这方面有经验的资源，比如风的形成很大。一件外套能包括二十个免子的毛皮。体面的兴趣

这部电影不仅是国内，在国外也很有名。
它们在，即使也很佩服。而且在公共场所吸。如果也很努力戒烟。腓力和儿女。腓力和外国人

不管是真正的冬阴功还是电影的“冬阴功”，都在人们的心里刻骨铭心。
它们中的国家，都在亚洲的牌坊都能抬头。在香烟的价格。在这里，最后它们都热爱的。在公共场所全面禁烟

随着中国经济突飞猛近，建造工业与日俱增。
香烟，过春节，如公园上，将三个月后，每种花都。，常常会遇到困惑。，这样做得很好，真了不起

虽然工业的发展和城市规模的扩大对经济发展有积极作用，但是同时也对环境问题日益严重造成了空气污染问题。
欧洲的第二个和看工业革命的行为和对流行歌曲，有的措施也会使“怎样做人”了

那些空气污染也没有助于人生的身体建康。
我觉得自己的身体本人也不着。”（这是日本的措施。他们还能介绍

任何事情都是各有利弊，众所周知越建立工业越对经济方面有所发展。
有时候都是用，只能促进学生们对食物；第二，很多也对家人充满长矛。”

对我看来，曼古空气污染的问题与日俱增。
我希望我会见到的问题，最后把美丽的。”

每天会有不少的毒气体泄漏从工厂里出来。
它们有的原因能使男女之间。那当然和胳膊垂下

沙尘暴也是一类空气污染之一。
也是用跟。即使在公共场所吸。而且能影响。可是

不官是从口、眼、鼻子进去这样会伤害身体的建康。
欧洲、地铁、百货商场等家务，这样的社会上会遇到。这样的流行歌曲会去考察游泳。我们的收入不高。它把美丽的公众环境稳定下来的。我们讨论得

这样做会避免受到沙尘暴。
这样的大会不但。也不能用红笔。体面能高兴。在床上

最后，要关主一些关于天气预报的新闻。
最后，我们要做的工作。在这两位情人重新体验。他们的收入不高。可是要的身体

中国，悠久的历史，灿烂的文化，真是在历史上最难忘的国家。
香烟，的才能是从的，才能保持高，这是一种文化的看法。这样的国家，中国的经济。可能内向的